In [1]:
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000
from pycorenlp import StanfordCoreNLP
import pandas as pd
import re
import fitz #pip install PyMuPDF
nlp_wrapper = StanfordCoreNLP('http://localhost:9000')

In [2]:
def extractText(file): 
    doc = fitz.open(file) 
    text = ""
    for page in doc: 
        text += str(page.getText())#.encode("utf8"))
    doc.close()
    return text

In [3]:
#open file .html
text = extractText("./bankstatementinput.pdf")
print(text)
text_split = text.split("\n")

SG40020000876 (08/2014) DBS Co. Reg. No. 196800306E GST Reg No: MR-8500180-3 / POSB Biz Reg No. 52880148A
Page 1 of 4
DBS Bank Ltd 
12 Marina Boulevard, Marina Bay Financial Centre Tower 3, Singapore 018982 
www.dbs.com.sg | www.posb.com.sg
For enquiries, please call our 24-Hour Hotline 
at 1800- 111 1111 (in Singapore) or (65) 6327 
2265 (outside Singapore)
S/N: EN05310800573439
CONSOLIDATED STATEMENT
GALI KARTHIK KUMAR 
TAIGER SINGAPORE PTE. LTD.
26B TEMPLE STREET
#03-00
SINGAPORE 058571
                     
ACCOUNT SUMMARY     |     As at 31 Aug 2018
DEPOSITS
MULTI CURRENCY
Account
Account Number
Balance
(S$ Equivalent)
DBS Multiplier Account
SGD 2,943.76
010-025010-2
 
2,943.76
TOTAL  DEPOSITS – CREDIT
2,943.76
PDS1.DFILE.SJSTCDMM.ONSH.ESTM.6-1.D180901T124036.01
(0585717)
Joh
JOHN RIEGER
KTGIF
12-145753-2
SG40020000876 (08/2014) DBS Co. Reg. No. 196800306E GST Reg No: MR-8500180-3 / POSB Biz Reg No. 52880148A
DBS Bank Ltd 
12 Marina Boulevard, Marina Bay Financial Centre Tower 3, 

In [4]:
def get_stanfordnlp(sentence):
    annot_doc = nlp_wrapper.annotate(sentence,
    properties={
        'annotators': 'ner, pos',
        'outputFormat': 'json',
        'timeout': 1000,
    })
    return annot_doc

In [5]:
name = []
suspect_date2 = []
suspect_account_number = []
address_suspect=[]
get_transaction=[]
suspect_account_number=[]

loc = ['COUNTRY','LOCATION','CITY','ORGANIZATION','PERSON']

for i in range (len(text_split)):
    annot_doc = get_stanfordnlp(text_split[i])
#     print(text_split[i])
    for sentence in annot_doc["sentences"]:
        m = len(sentence['entitymentions'])
        if m!=0:
            for j in range(m):
#                 print(sentence['entitymentions'][j]['ner'])
                if sentence['entitymentions'][j]['ner'] == "PERSON":
                    suspect_name = []
#                     print(sentence['tokens'])
                    word = text_split[i].split()
                    n = len(word)
                    k = 0
                    for w in word:
                        annot_tok = get_stanfordnlp(w)
                        for token in annot_tok['sentences']:
                            if (token['tokens'][0]['ner']) == "PERSON":
                                k+=1
                    if n == k:
#                         print(text_split[i])
#                     suspect_name.append(w)
                        name.append(text_split[i])
                #get date
                if sentence['entitymentions'][j]['ner'] ==  "DATE":
                    n = len(sentence['tokens'])
                    suspect_date=[]
#                     y = len(sentence['tokens']['normalizedNER)
                    y=0
                    for k in range (n):
#                         print(sentence['tokens'][k]['normalizedNER'])
#                         y = len(sentence['tokens'][k]['normalizedNER'])
                        if sentence['tokens'][k]['ner'] == "DATE":
                            suspect_date.append(sentence['tokens'][k]['word'])
#                             print(sentence['tokens'][k]['word'],"---",sentence['tokens'][k]['ner'])
                            if len(sentence['tokens'][k]['word'])==4:
#                                 print(sentence['tokens'][k]['word']) #asumsi all of date never be year
                                y+=1
                    if len(suspect_date)>1 and y<2: # y is amount of year.
                        suspect_date2.append(" ".join(suspect_date))
                #get number for account
                if sentence['entitymentions'][j]['ner'] ==  "NUMBER":
                    n = len (sentence['tokens'])
                    
                    collection_list=[]
#                     print(n)
                    for k in range(n):
                        if(sentence['tokens'][k]['ner']=="NUMBER"):
                            y=len(sentence['tokens'][k]['word'])
                            if y >10: #got number length > 10
#                             collection_list.append(sentence['tokens'][k]['ner'])
#                     for m in loc:
#                         if m not in collection_list:
#                             print(collection_list)
                                suspect_account_number.append(sentence['tokens'][k]['word'])
    #because many statement bank paper, the address after name.        
    for n in name:
        if n in text_split[i]:
#             print(text_split[i])
            j=i+1
            suspect_address=[]
            while(j<len(text_split)):
                temp=False
                annot_doc = get_stanfordnlp(text_split[j])
                suspect_address.append(text_split[j])
#                 print(text_split[j])
                for sentence in annot_doc["sentences"]:
            #         for word in sentence["tokens"]:
            #         print(sentence)
            #         print("***")
                    m = len(sentence['entitymentions'])
                    if m!=0:
                        for k in range(m):
#                             print(sentence['entitymentions'][k]['ner'])
                            if sentence['entitymentions'][k]['ner'] == "COUNTRY":
                                temp = True
                if temp == True:
                    break
                j+=1
            address_suspect.append(" ".join(suspect_address))
#     info=[]
        
#                     suspect_account_number.append(i)
#     print("_--------_")
#         print (sentence["word"] + "=>" + sentence["ner"])


In [6]:
print(name)
print(suspect_date2)
print(suspect_account_number)
print(address_suspect)
# print(suspect_account_number)

['GALI KARTHIK KUMAR ', 'Joh', 'JOHN RIEGER', 'KTGIF']
['31 Aug 2018', '31 Aug 2018', '28 Aug', '30 Aug', '30 Aug', '31 Aug', '31 Aug']
['12-145753-2', '010-025010-2']
['TAIGER SINGAPORE PTE. LTD. 26B TEMPLE STREET #03-00 SINGAPORE 058571', 'JOHN RIEGER KTGIF 12-145753-2 SG40020000876 (08/2014) DBS Co. Reg. No. 196800306E GST Reg No: MR-8500180-3 / POSB Biz Reg No. 52880148A DBS Bank Ltd  12 Marina Boulevard, Marina Bay Financial Centre Tower 3, Singapore 018982 ', 'KTGIF 12-145753-2 SG40020000876 (08/2014) DBS Co. Reg. No. 196800306E GST Reg No: MR-8500180-3 / POSB Biz Reg No. 52880148A DBS Bank Ltd  12 Marina Boulevard, Marina Bay Financial Centre Tower 3, Singapore 018982 ', '12-145753-2 SG40020000876 (08/2014) DBS Co. Reg. No. 196800306E GST Reg No: MR-8500180-3 / POSB Biz Reg No. 52880148A DBS Bank Ltd  12 Marina Boulevard, Marina Bay Financial Centre Tower 3, Singapore 018982 ']


In [7]:
#get transaction bank
get_transaction=[]
for i in range(len(text_split)):
    if text_split[i] in suspect_date2:
        print(text_split[i])
        j=i+1
        info=[]
        info.append(text_split[i])
        
        while(j<len(text_split)):
#             temp=False
            if text_split[j] not in suspect_date2:
                if text_split[j].lower() in 'total':
                    break
                else:
                    info.append(text_split[j])
            else:
                break
            j+=1
        get_transaction.append(info)
get_transaction

28 Aug
30 Aug
30 Aug
31 Aug
31 Aug


[['28 Aug', 'Quick Cheque Deposit', '3,471.26', '3,471.26'],
 ['30 Aug',
  'Point-of-Sale Transaction',
  'TRANSIT LINK PTE LTD              ',
  '20.00'],
 ['30 Aug',
  'Point-of-Sale Transaction',
  'S & S LINKERS PTE LTD             ',
  '465.00',
  '2,986.26'],
 ['31 Aug',
  'Point-of-Sale Transaction',
  'GAYATRI RESTAURANT                ',
  '26.50'],
 ['31 Aug',
  'Point-of-Sale Transaction',
  'FOCUS NETWORK AGENCIES (S) PTE LTD',
  '16.00',
  '2,943.76']]

In [8]:
date=[]
description=[]
amount=[]
type_=[]
for i in get_transaction:
    get_date=[]
    get_desc=[]
    get_amount=[]
    get_info=[]
    for j in i:
        print(j)
        annot_doc = get_stanfordnlp(j)
        for ls in annot_doc['sentences']:
            if ls["tokens"][0]['ner'] == "DATE":
                get_date.append(j)
            elif ls["tokens"][0]['ner'] == "NUMBER":
                get_amount.append(j)
            else:
                get_desc.append(j)
    date.append(get_date)
    description.append(get_desc)
    amount.append(get_amount)


28 Aug
Quick Cheque Deposit
3,471.26
3,471.26
30 Aug
Point-of-Sale Transaction
TRANSIT LINK PTE LTD              
20.00
30 Aug
Point-of-Sale Transaction
S & S LINKERS PTE LTD             
465.00
2,986.26
31 Aug
Point-of-Sale Transaction
GAYATRI RESTAURANT                
26.50
31 Aug
Point-of-Sale Transaction
FOCUS NETWORK AGENCIES (S) PTE LTD
16.00
2,943.76


In [9]:
dc = zip(date, description,amount)
df_data_transaction = pd.DataFrame(dc, columns=['Date','Desc','Amount'])
print("Name : ",name[0])
print("Address: ",address_suspect[0]) 
print("Account Number: ",suspect_account_number[0])
print("Statement Date: ",suspect_date2[0])
   
# print("", get_transaction)
df_data_transaction

Name :  GALI KARTHIK KUMAR 
Address:  TAIGER SINGAPORE PTE. LTD. 26B TEMPLE STREET #03-00 SINGAPORE 058571
Account Number:  12-145753-2
Statement Date:  31 Aug 2018


,Date,Desc,Amount
0,[28 Aug],[Quick Cheque Deposit],"[3,471.26, 3,471.26]"
1,[30 Aug],"[Point-of-Sale Transaction, TRANSIT LINK PTE L...",[20.00]
2,[30 Aug],"[Point-of-Sale Transaction, S & S LINKERS PTE ...","[465.00, 2,986.26]"
3,[31 Aug],"[Point-of-Sale Transaction, GAYATRI RESTAURANT...",[26.50]
4,[31 Aug],"[Point-of-Sale Transaction, FOCUS NETWORK AGEN...","[16.00, 2,943.76]"
